### Import and config

In [1]:
# Imports
import os
import logging
from datetime import timezone, timedelta

import polyline

from dotenv import load_dotenv

import pandas as pd
import numpy as np

from sqlalchemy import create_engine, text, Integer, Float, String, Boolean, DateTime, Interval, Text, BigInteger

# Geopy - Nominatim
import json, time
from tqdm import tqdm
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Configuration
load_dotenv()

# DB
DB_URI = os.getenv('DB_URI')

# Bronze tables
TARGET_B_SCHEMA = os.getenv('TARGET_B_SCHEMA')
ACTIVITIES_B_TABLE = os.getenv('ACTIVITIES_B_TABLE')
DETAILS_B_TABLE = os.getenv('DETAILS_B_TABLE')
KUDOS_B_TABLE = os.getenv('KUDOS_B_TABLE')
ZONES_B_TABLE = os.getenv('ZONES_B_TABLE')

# Silver tables
TARGET_S_SCHEMA = os.getenv('TARGET_S_SCHEMA')
ACTIVITIES_S_TABLE = os.getenv('ACTIVITIES_S_TABLE')
BEST_EFFORTS_S_TABLE = os.getenv('BEST_EFFORTS_S_TABLE')
GEAR_S_TABLE = os.getenv('GEAR_S_TABLE')
LAPS_S_TABLE = os.getenv('LAPS_S_TABLE')
MAPS_S_TABLE = os.getenv('MAPS_S_TABLE')
SEGMENTS_S_TABLE = os.getenv('SEGMENTS_S_TABLE')
SEGMENTS_EFFORTS_S_TABLE = os.getenv('SEGMENTS_EFFORTS_S_TABLE')
LOCATIONS_S_TABLE = os.getenv('LOCATIONS_S_TABLE')
KUDOS_S_TABLE = os.getenv('KUDOS_S_TABLE')
ZONES_S_TABLE = os.getenv('ZONES_S_TABLE')
RELATIVE_EFFORT_S_TABLE = os.getenv('RELATIVE_EFFORT_S_TABLE')

# Other
LOG_LEVEL = os.getenv('LOG_LEVEL')
NO_GEAR_ID = 'x00000000'

# Geopy - Nominatim
PRECISION = int(os.getenv('PRECISION'))
CACHE_PATH = os.getenv('CACHE_PATH')
USER_AGENT = os.getenv('USER_AGENT')

logging.basicConfig(
    level=getattr(logging, LOG_LEVEL.upper(), logging.INFO),
    format="%(asctime)s | %(levelname)s | %(message)s"
)

pd.set_option('display.max_columns', None)

### DB names validation

In [2]:
REQUIRED_DB_ENV = ['DB_URI', 'TARGET_B_SCHEMA', 'ACTIVITIES_B_TABLE', 'DETAILS_B_TABLE','KUDOS_B_TABLE', 'ZONES_B_TABLE', 'TARGET_S_SCHEMA', 'ACTIVITIES_S_TABLE', 'BEST_EFFORTS_S_TABLE', 'GEAR_S_TABLE', 'LAPS_S_TABLE', 'MAPS_S_TABLE', 'SEGMENTS_S_TABLE', 'SEGMENTS_EFFORTS_S_TABLE', 'ZONES_S_TABLE', 'RELATIVE_EFFORT_S_TABLE']
missing_db_env = [env for env in REQUIRED_DB_ENV if not os.getenv(env)]
if missing_db_env:
  raise RuntimeError(f"Missing env variables: {', '.join(missing_db_env)}.")

### Request data from `bronze.activities_details` and `bronze.kudos`

In [ ]:
engine = create_engine(
  DB_URI, 
  pool_pre_ping=True, 
  pool_size=5, 
  max_overflow=10
)
logging.info("Connection established")

In [ ]:
with engine.begin() as conn:
  activities_details_df = pd.read_sql(text(f"SELECT * FROM {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}"), conn)
  kudos_df = pd.read_sql(text(f"SELECT * FROM {TARGET_B_SCHEMA}.{KUDOS_B_TABLE}"), conn)
  activities_zones_df = pd.read_sql(text(f"SELECT * FROM {TARGET_B_SCHEMA}.{ZONES_B_TABLE}"), conn)
logging.info(f"Data from {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}, {TARGET_B_SCHEMA}.{KUDOS_B_TABLE} and {TARGET_B_SCHEMA}.{ZONES_B_TABLE} downloaded.")

In [ ]:
activities_details_df.head()

In [ ]:
kudos_df.head()

In [ ]:
activities_zones_df.head()

### Separate tables setup

In [8]:
dataframe_columns = {
  'activities' : [
    'id',
    'name',
    'distance',
    'moving_time',
    'elapsed_time',
    'total_elevation_gain',
    'type',
    'sport_type',
    'workout_type',
    'start_date',
    'start_date_local',
    'timezone',
    'achievement_count',
    'kudos_count',
    'comment_count',
    'athlete_count',
    'photo_count',
    'trainer',
    'commute',
    'manual',
    'visibility',
    'start_latlng',
    'end_latlng',
    'average_speed',
    'max_speed',
    'average_cadence',
    'average_watts',
    'max_watts',
    'weighted_average_watts',
    'has_heartrate',
    'average_heartrate',
    'max_heartrate',
    'elev_high',
    'elev_low',
    'pr_count',
    'total_photo_count',
    'suffer_score',
    'description',
    'calories',
    'device_name',
    'map_id',
    'gear_id'],
  'maps' : [
    'map_id',
    'map_polyline',
    'map_summary_polyline'],
  'gear' : [
    'gear_id',
    'gear_name',
    'gear_distance',
    'gear_converted_distance',
    'start_date',
    'start_date_local'],
  'segment_efforts' : [
    'id',
    'name',
    'elapsed_time',
    'moving_time',
    'start_date',
    'start_date_local',
    'distance',
    'start_index',
    'end_index',
    'average_cadence',
    'device_watts',
    'average_watts',
    'average_heartrate',
    'max_heartrate',
    'pr_rank',
    'achievements',
    'visibility',
    'kom_rank',
    'hidden',
    'activity_id',
    'segment_id'],
  'segments' : [
    'segment_id',
    'segment_name',
    'segment_activity_type',
    'segment_distance',
    'segment_average_grade',
    'segment_maximum_grade',
    'segment_elevation_high',
    'segment_elevation_low',
    'segment_start_latlng',
    'segment_end_latlng',
    'segment_elevation_profile',
    'segment_elevation_profiles',
    'segment_climb_category',
    'segment_private',
    'segment_hazardous',
    'segment_starred'],
  'laps' : [
    'id',
    'name',
    'elapsed_time',
    'moving_time',
    'start_date',
    'start_date_local',
    'distance',
    'average_speed',
    'max_speed',
    'lap_index',
    'split',
    'start_index',
    'end_index',
    'total_elevation_gain',
    'average_cadence',
    'device_watts',
    'average_watts',
    'average_heartrate',
    'max_heartrate',
    'pace_zone',
    'activity_id'],
  'best_efforts' : [
    'id',
    'activity_id',
    'name',
    'elapsed_time',
    'moving_time',
    'start_date',
    'start_date_local',
    'distance',
    'pr_rank',
    'achievements',
    'start_index',
    'end_index']
}

### Spliting data into tables

In [9]:
def select_cols(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
  """
  Select only the specified columns from a DataFrame if they exist.

  Parameters
  ----------
  df : pd.DataFrame
      The input DataFrame.
  cols : list of str
      List of column names to select.

  Returns
  -------
  pd.DataFrame
      A new DataFrame containing only the specified columns that exist 
      in the input DataFrame. If none of the columns exist, 
      an empty DataFrame is returned.
  """
  
  existing = [c for c in cols if c in df.columns]
  
  return df[existing].copy() if existing else pd.DataFrame()

def explode_normalize_json(df: pd.DataFrame, col: str, id_col: str | None = None, id_name: str | None = None) -> pd.DataFrame:
  """
  Explode a list-like column into multiple rows and normalize nested JSON/dict objects 
  into a flat tabular structure.

  This function is useful for columns containing arrays of JSON objects 
  (e.g. laps, segment efforts). Each element of the list becomes a separate row, 
  and nested fields are flattened into individual columns. Optionally, 
  a parent identifier column can be retained/renamed to act as a foreign key.

  Parameters
  ----------
  df : pd.DataFrame
      Input DataFrame containing the column to explode.
  col : str
      Name of the column with list- or dict-like values to explode and normalize.
  id_col : str, optional
      Name of the column in the input DataFrame to keep as a parent identifier.
      If provided, it will be included in the output.
  id_name : str, optional
      If provided together with `id_col`, renames the identifier column 
      in the result (e.g. from "id" to "activity_id").

  Returns
  -------
  pd.DataFrame
      A new DataFrame where:
        * each list element from `col` is a separate row,
        * nested JSON/dict objects are flattened into columns with names joined by "_",
        * the parent identifier (`id_col`) is preserved and optionally renamed.
      If `col` is missing or contains only null/empty values, 
      an empty DataFrame is returned.
  """

  if col not in df.columns:
    return pd.DataFrame()
  
  base_cols = [col]

  if id_col and id_col in df.columns:
    base_cols.insert(0, id_col)

  base = df[base_cols].copy()
  exploded = base.explode(col, ignore_index=True)
  values = exploded[col].dropna()

  if values.empty:
    return pd.DataFrame()
  
  norm = pd.json_normalize(values, sep='_')
  out = exploded.loc[values.index].drop(columns=[col]).reset_index(drop=True)
  res = pd.concat([out.reset_index(drop=True), norm.reset_index(drop=True)], axis=1)
  
  if id_col and id_name and id_col in df.columns:
    res = res.rename(columns={id_col: id_name})
  return res

In [ ]:
# Activities
activities_cols = dataframe_columns['activities']
activities_df = select_cols(activities_details_df, activities_cols)
logging.info("DataFrame 'activities_df' created.")

# Maps
maps_cols = dataframe_columns['maps']
maps_df = select_cols(activities_details_df, maps_cols)
logging.info("DataFrame 'maps_df' created.")

# Gear
gear_cols = dataframe_columns['gear']
gear_df = select_cols(activities_details_df, gear_cols)
logging.info("DataFrame 'gear_df' created.")

# Segment efforts
seg_eff_cols = dataframe_columns['segment_efforts']
segments_eff_df = explode_normalize_json(activities_details_df, 'segment_efforts')
segments_eff_df = select_cols(segments_eff_df, seg_eff_cols)
logging.info("DataFrame 'segments_eff_df' created.")

# Segments
seg_cols = dataframe_columns['segments']
segments_df = explode_normalize_json(activities_details_df, 'segment_efforts')
segments_df = select_cols(segments_df, seg_cols)
logging.info("DataFrame 'segments_df' created.")

# Laps
lap_cols = dataframe_columns['laps']
laps_df = explode_normalize_json(activities_details_df, 'laps')
laps_df = select_cols(laps_df, lap_cols)
logging.info("DataFrame 'laps_df' created.")

# Best efforts
best_eff_cols = dataframe_columns['best_efforts']
best_eff_df = explode_normalize_json(activities_details_df, 'best_efforts')
best_eff_df = select_cols(best_eff_df, best_eff_cols)
logging.info("DataFrame 'best_eff_df' created.")

# All dataframes in dictionary
dataframes = {
    "activities": activities_df,
    "maps": maps_df,
    "gear": gear_df,
    "segment_efforts": segments_eff_df,
    "segments": segments_df,
    "laps": laps_df,
    "best_efforts": best_eff_df
}

### Activities Dataframe

In [11]:
def speed_to_pace_str(speed: float) -> str | None:
  """
  Convert speed in meters per second to running pace as a string.

  Parameters
  ----------
  speed : float
      Speed value in meters per second. Must be greater than zero.

  Returns
  -------
  str or None
      Running pace in the format "M:SS" representing minutes per kilometer.
      For example, "5:32" means 5 minutes and 32 seconds per kilometer.
      Returns None if the speed is less than or equal to zero.
  """

  if speed <= 0:
    return None
  
  seconds = 1000/speed
  minutes = int(seconds // 60)
  sec = int(round(seconds % 60))

  if sec == 60:
    minutes += 1
    sec = 0

  return f"{minutes}:{sec:02d}"

def speed_to_pace_float(speed: float) -> float | None:
  """
  Convert speed in meters per second to running pace as a float.

  Parameters
  ----------
  speed : float
      Speed value in meters per second. Must be greater than zero.

  Returns
  -------
  float or None
      Running pace in minutes per kilometer, represented as a float.
      For example, 5.53 means approximately 5 minutes and 32 seconds per kilometer.
      Returns None if the speed is less than or equal to zero.
  """

  if speed <= 0:
    return None
  
  return 1000 / speed / 60

def extract_timedelta(time: pd.Series) -> pd.Series:
  """
  Convert a Series of numeric values (seconds) into timedeltas.

  Parameters
  ----------
  time : pd.Series
      Series containing durations expressed in seconds (int/float). 
      Null values are preserved as None.

  Returns
  -------
  pd.Series
      Series of Python ``datetime.timedelta`` objects. 
      Each element corresponds to the given number of seconds or None if missing.
  """

  return pd.Series([(timedelta(seconds=int(t)) if pd.notnull(t) else None) for t in time], dtype="object")

def extract_latlng(latlng: pd.Series) -> pd.DataFrame:
  """
  Split a Series of latitude/longitude pairs into a DataFrame with separate columns.

  Parameters
  ----------
  latlng : pd.Series
      Series where each element is expected to be a list or tuple of length 2 
      (latitude, longitude). If the element is not a valid pair, it is replaced 
      with [None, None].

  Returns
  -------
  pd.DataFrame
      DataFrame with two columns:
        * first column: latitude
        * second column: longitude
      The index is preserved from the input Series.
  """
  latlng = latlng.apply(
    lambda row: row if isinstance(row, (list, tuple)) and len(row) == 2 else [None, None]
  )
  return pd.DataFrame(latlng.tolist(), index=latlng.index)

def etc_gmt_from_offset(minutes: int) -> str:
    """
    Convert a UTC offset (in minutes) to an IANA fixed-offset zone name ``Etc/GMT±N``.

    Parameters
    ----------
    minutes : int
        Offset from UTC in minutes.
        Positive values mean UTC+ (east of Greenwich), negative mean UTC− (west).

    Returns
    -------
    str
        IANA timezone name. For whole-hour offsets the format is ``Etc/GMT±H``,
        e.g. ``Etc/GMT-2`` for +120 minutes and ``Etc/GMT+5`` for −300 minutes.
        For non-hour offsets, minutes are included, e.g. ``Etc/GMT-2:30``.

    Notes
    -----
    - The ``Etc/GMT`` naming convention uses an **inverted sign** relative to ISO 8601:
      ``UTC+02:00 → Etc/GMT-2`` and ``UTC-05:00 → Etc/GMT+5``.
    - ``Etc/GMT`` zones are fixed-offset and **do not observe DST**.

    Examples
    --------
    >>> etc_gmt_from_offset(120)
    'Etc/GMT-2'
    >>> etc_gmt_from_offset(-300)
    'Etc/GMT+5'
    >>> etc_gmt_from_offset(150)
    'Etc/GMT-2:30'
    """

    sign = '-' if minutes > 0 else '+'
    h, m = divmod(abs(minutes), 60)
    return f"Etc/GMT{sign}{h}" if m == 0 else f"Etc/GMT{sign}{h}:{m:02d}"

def create_datetime_tz_cols(df: pd.DataFrame, date_col: str, date_col_local: str) -> pd.DataFrame:
  """
  Derive a UTC timestamp and a fixed-offset timezone name from UTC and local datetimes.

  The function parses a UTC datetime column and a corresponding local datetime column
  (both representing the same instant), computes the offset in minutes
  ``local - utc``, and maps that offset to a fixed-offset IANA zone name
  using ``Etc/GMT±H[:MM]``. It returns a DataFrame with the UTC timestamp and
  the derived timezone name.

  Parameters
  ----------
  df : pd.DataFrame
      Input DataFrame containing the UTC and local datetime columns.
  date_col : str
      Name of the column with UTC datetimes (string or datetime-like).
      Values are parsed to a tz-aware UTC dtype.
  date_col_local : str
      Name of the column with local datetimes (string or datetime-like).
      Values are used only to infer the UTC offset.

  Returns
  -------
  pd.DataFrame
      A DataFrame with two columns:
        * ``start_date_utc_dt`` — tz-aware UTC timestamp (dtype ``datetime64[ns, UTC]``),
        * ``tz`` — fixed-offset IANA zone name in the ``Etc/GMT`` family
          (e.g., ``"Etc/GMT-2"`` for UTC+02:00, ``"Etc/GMT+5"`` for UTC−05:00).

  Notes
  -----
  - ``Etc/GMT`` zones are fixed offsets and **do not observe DST**. The sign is
    intentionally inverted by IANA naming convention: UTC+02:00 → ``Etc/GMT-2``.
  - Both columns must refer to the same moment in time; otherwise the inferred
    offset (and thus ``tz``) will be incorrect.

  Raises
  ------
  KeyError
      If ``date_col`` or ``date_col_local`` is missing in ``df``.
  ValueError
      If datetime parsing fails.
  """
  
  if date_col not in df.columns or date_col_local not in df.columns:
        raise KeyError(f"Missing required columns: {date_col}, {date_col_local}")

  temp_df = pd.DataFrame()
  temp_df["start_date_utc_dt"] = pd.to_datetime(df[date_col])
  temp_df["start_date_local_dt"] = pd.to_datetime(df[date_col_local])
  temp_df["utc_offset"] = (temp_df["start_date_local_dt"] - temp_df["start_date_utc_dt"]).dt.total_seconds() / 60

  temp_df["tz"] = temp_df["utc_offset"].apply(etc_gmt_from_offset)
  
  return temp_df[["start_date_utc_dt", "start_date_local_dt", "tz"]]

activity_details

In [12]:
activities_df.loc[:, "moving_time_td"]  = extract_timedelta(activities_df["moving_time"])
activities_df.loc[:, "elapsed_time_td"] = extract_timedelta(activities_df["elapsed_time"])

activities_df[["start_date_utc_dt", "start_date_local_dt", "local_timezone"]] = create_datetime_tz_cols(activities_df, "start_date", "start_date_local")

activities_df[["start_lat", "start_lng"]]  = extract_latlng(activities_df["start_latlng"])
activities_df[["end_lat", "end_lng"]] = extract_latlng(activities_df["end_latlng"])

is_run = activities_df['type'] == 'Run'

activities_df.loc[is_run, 'average_cadence'] = activities_df['average_cadence'].apply(lambda x: x * 2)

activities_df.loc[is_run, 'avg_pace_str'] = activities_df['average_speed'].apply(speed_to_pace_str)
activities_df.loc[is_run, 'avg_pace_float'] = activities_df['average_speed'].apply(speed_to_pace_float)

activities_df.loc[is_run, 'max_pace_str'] = activities_df['max_speed'].apply(speed_to_pace_str)
activities_df.loc[is_run, 'max_pace_float'] = activities_df['max_speed'].apply(speed_to_pace_float)

activities_df['gear_id'] = activities_df['gear_id'].fillna(NO_GEAR_ID)


In [13]:
activities_df['gear_id'] = np.where(
    (activities_df['start_date_utc_dt'] < '2020-01-01') &
    (activities_df['gear_id'] == 'g9239745'),
    NO_GEAR_ID,
    activities_df['gear_id']
)

In [14]:
activities_cols_clean = [
    'id',
    'name',
    'start_date_utc_dt',
    'start_date_local_dt',
    'local_timezone',
    'distance',
    'moving_time',
    'moving_time_td',
    'elapsed_time',
    'elapsed_time_td',
    'total_elevation_gain',
    'elev_low',
    'elev_high',
    'type',
    'sport_type',
    'workout_type',
    'achievement_count',
    'kudos_count',
    'comment_count',
    'athlete_count',
    'photo_count',
    'trainer',
    'commute',
    'manual',
    'visibility',
    'average_speed',
    'avg_pace_str',
    'avg_pace_float',
    'max_speed',
    'max_pace_str',
    'max_pace_float',
    'average_cadence',
    'average_watts',
    'max_watts',
    'weighted_average_watts',
    'has_heartrate',
    'average_heartrate',
    'max_heartrate',
    'pr_count',
    'total_photo_count',
    'suffer_score',
    'description',
    'calories',
    'device_name',
    'start_lat',
    'start_lng',
    'map_id',
    'gear_id'
]
activities_df = activities_df[activities_cols_clean]
activities_df = activities_df.sort_values(by='start_date_utc_dt', ascending=False)

In [ ]:
activities_df.head()

Load to PostgreSQL will be made after extracting location from coordinates

## Maps Dataframe

In [16]:
maps_df.columns = maps_df.columns.str.replace("^map_", "", regex=True)

In [17]:
maps_cols_clean = [
  'id',
  'polyline',
  'summary_polyline'
]
maps_df = maps_df[maps_cols_clean]

In [18]:
maps_df['latlng'] = maps_df.apply(
  lambda row: polyline.decode(row['polyline']) if isinstance(row['polyline'], str) else [], axis=1
)
maps_df = maps_df.explode('latlng', ignore_index=False)
maps_df['point_id'] = maps_df.groupby(level=0).cumcount()
maps_df[['lat','lng']] = extract_latlng(maps_df['latlng'])
maps_df = maps_df.drop(columns=['polyline', 'summary_polyline', 'latlng'])

In [ ]:
maps_df

In [ ]:
maps_df_dtype_map = {
    "id": String,
    "point_id": Integer,
    "lat": Float,
    "lng": Float
}

with engine.begin() as conn:
    conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {TARGET_S_SCHEMA};"))

logging.warning(f"Whole table {TARGET_S_SCHEMA}.{MAPS_S_TABLE} will be overwritten.")

maps_df.to_sql(
    name=MAPS_S_TABLE,
    schema=TARGET_S_SCHEMA,
    con=engine,
    if_exists="replace",
    index=False,
    dtype=maps_df_dtype_map,
    method="multi",
    chunksize=1000
)

## Gear Dataframe

In [21]:
gear_df.columns = gear_df.columns.str.replace("^gear_", "", regex=True)
gear_df['id'] = gear_df['id'].fillna(NO_GEAR_ID)
gear_df['name'] = gear_df['name'].fillna('No gear')
gear_df = gear_df.fillna(0)
gear_df = gear_df.rename(columns={'distance' : 'distance_m', 'converted_distance' : 'distance_km'})
gear_df[["start_date_utc_dt", "start_date_local_dt", "local_timezone"]] = create_datetime_tz_cols(gear_df, "start_date", "start_date_local")
gear_df = (
  gear_df.sort_values(by='start_date_utc_dt', ascending=False)
  .drop_duplicates(subset=["id"], keep="first")
  .reset_index(drop=True)
)
gear_df = gear_df[['id', 'name', 'distance_m', 'distance_km']]

In [ ]:
gear_df

In [ ]:
gear_df_dtype_map = {
    "id": String,
    "name": String,
    "distance_m": Float,
    "distance_km": Float,
}

with engine.begin() as conn:
    conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {TARGET_S_SCHEMA};"))

logging.warning(f"Whole table {TARGET_S_SCHEMA}.{GEAR_S_TABLE} will be overwritten.")

gear_df.to_sql(
    name=GEAR_S_TABLE,
    schema=TARGET_S_SCHEMA,
    con=engine,
    if_exists="replace",
    index=False,
    dtype=gear_df_dtype_map,
    method="multi",
    chunksize=1000
)

## Segments efforts Dataframe

In [24]:
segments_types_df = segments_df[['segment_id', 'segment_activity_type']].copy()
segments_types_df.drop_duplicates(inplace=True)

segments_eff_df = pd.merge(segments_eff_df, segments_types_df, on='segment_id', how='left')
segments_eff_df = pd.merge(segments_eff_df.drop(columns="achievements"), explode_normalize_json(segments_eff_df, 'achievements', 'id'), on='id', how='left')

segments_eff_df.loc[:, "moving_time_td"]  = extract_timedelta(segments_eff_df["moving_time"])
segments_eff_df.loc[:, "elapsed_time_td"] = extract_timedelta(segments_eff_df["elapsed_time"])

segments_eff_df[["start_date_utc_dt", "start_date_local_dt", "local_timezone"]] = create_datetime_tz_cols(segments_eff_df, "start_date", "start_date_local")

is_run = segments_eff_df['segment_activity_type'] == 'Run'

segments_eff_df.loc[is_run, 'average_cadence'] = segments_eff_df['average_cadence'].apply(lambda x: x * 2)

In [25]:
segments_eff_cols_clean = [
  'id',
  'name',
  'start_date_utc_dt',
  'start_date_local_dt',
  'local_timezone',
  'distance',
  'moving_time',
  'moving_time_td',
  'elapsed_time',
  'elapsed_time_td',
  'average_cadence',
  'device_watts',
  'average_watts',
  'average_heartrate',
  'max_heartrate',
  'pr_rank',
  'visibility',
  'kom_rank',
  'hidden',
  'rank',
  'type',
  'activity_id',
  'segment_id'
]
segments_eff_df = segments_eff_df[segments_eff_cols_clean]

In [ ]:
segments_eff_df.head()

In [ ]:
segments_eff_df_dtype_map = {
    "id": BigInteger,
    "name": String,
    "start_date_utc_dt": DateTime(timezone=False),
    "start_date_local_dt": DateTime(timezone=False),
    "distance": Float,
    "moving_time": Integer,
    "moving_time_td": Interval,
    "elapsed_time": Integer,
    "elapsed_time_td": Interval,
    "average_cadence": Float,
    "device_watts": Boolean,
    "average_watts": Float,
    "average_heartrate": Float,
    "max_heartrate": Float,
    "pr_rank": Integer,
    "visibility": String,
    "kom_rank": Integer,
    "hidden": Boolean,
    "rank": Integer,
    "type": String,
    "activity_id": BigInteger,
    "segment_id": BigInteger
}

with engine.begin() as conn:
    conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {TARGET_S_SCHEMA};"))

logging.warning(f"Whole table {TARGET_S_SCHEMA}.{SEGMENTS_EFFORTS_S_TABLE} will be overwritten.")

segments_eff_df.to_sql(
    name=SEGMENTS_EFFORTS_S_TABLE,
    schema=TARGET_S_SCHEMA,
    con=engine,
    if_exists="replace",
    index=False,
    dtype=segments_eff_df_dtype_map,
    method="multi",
    chunksize=1000
)

## Segments Dataframe

In [28]:
segments_df.columns = segments_df.columns.str.replace("^segment_", "", regex=True)

segments_df[["start_lat", "start_lng"]]  = extract_latlng(segments_df["start_latlng"])
segments_df[["end_lat", "end_lng"]] = extract_latlng(segments_df["end_latlng"])

segments_df = segments_df.drop(columns=["start_latlng", "end_latlng"])
segments_df = segments_df.drop_duplicates()

In [ ]:
segments_df.head()

In [ ]:
segments_df[segments_df['id'] == 20350088]

Load to PostgreSQL will be made after extracting location from coordinates

## Laps Dataframe

In [31]:
laps_types_df = activities_df[['id', 'type']].copy()
laps_types_df.drop_duplicates(inplace=True)
laps_df = pd.merge(laps_df, laps_types_df, left_on='activity_id', right_on='id', how='left')

In [32]:
laps_df.loc[:, "moving_time_td"]  = extract_timedelta(laps_df["moving_time"])
laps_df.loc[:, "elapsed_time_td"] = extract_timedelta(laps_df["elapsed_time"])

laps_df[["start_date_utc_dt", "start_date_local_dt", "local_timezone"]] = create_datetime_tz_cols(laps_df, "start_date", "start_date_local")

is_run = laps_df['type'] == 'Run'

laps_df.loc[is_run, 'average_cadence'] = laps_df['average_cadence'].apply(lambda x: x * 2)

laps_df.loc[is_run, 'avg_pace_str'] = laps_df['average_speed'].apply(speed_to_pace_str)
laps_df.loc[is_run, 'avg_pace_float'] = laps_df['average_speed'].apply(speed_to_pace_float)

laps_df.loc[is_run, 'max_pace_str'] = laps_df['max_speed'].apply(speed_to_pace_str)
laps_df.loc[is_run, 'max_pace_float'] = laps_df['max_speed'].apply(speed_to_pace_float)


In [33]:
laps_cols_df_clean = [
    'id_x',
    'name',
    'lap_index',
    'split',
    'start_date_utc_dt',
    'start_date_local_dt',
    'local_timezone',
    'distance',
    'moving_time',
    'moving_time_td',
    'elapsed_time',
    'elapsed_time_td',
    'total_elevation_gain',
    'type',
    'average_speed',
    'avg_pace_str',
    'avg_pace_float',
    'pace_zone',
    'max_speed',
    'max_pace_str',
    'max_pace_float',
    'average_cadence',
    'device_watts',
    'average_watts',
    'average_heartrate',
    'max_heartrate',
    'activity_id'
]

laps_df = laps_df[laps_cols_df_clean]
laps_df = laps_df.rename(columns={'id_x': 'id'})

In [ ]:
laps_df.head()

In [ ]:
laps_df_dtype_map = {
"id": BigInteger,
"name": String,
"lap_index": Integer,
"split": Integer,
"start_date_utc_dt": DateTime(timezone=False),
"start_date_local_dt": DateTime(timezone=False),
'local_timezone' : String,
"distance": Float,
"moving_time": Integer,
"moving_time_td": Interval,
"elapsed_time": Integer,
"elapsed_time_td": Interval,
"total_elevation_gain": Float,
"type": String,
"average_speed": Float,
"avg_pace_str": String,
"avg_pace_float": Float,
"pace_zone": Float,
"max_speed": Float,
"max_pace_str": String,
"max_pace_float": Float,
"average_cadence": Float,
"device_watts": Boolean,
"average_watts": Float,
"average_heartrate": Float,
"max_heartrate": Float,
"activity_id": BigInteger,
}

with engine.begin() as conn:
    conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {TARGET_S_SCHEMA};"))

logging.warning(f"Whole table {TARGET_S_SCHEMA}.{LAPS_S_TABLE} will be overwritten.")

laps_df.to_sql(
    name=LAPS_S_TABLE,
    schema=TARGET_S_SCHEMA,
    con=engine,
    if_exists="replace",
    index=False,
    dtype=laps_df_dtype_map,
    method="multi",
    chunksize=1000
)


## Best efforts Dataframe

In [36]:

best_eff_df.loc[:, "moving_time_td"]  = extract_timedelta(best_eff_df["moving_time"])
best_eff_df.loc[:, "elapsed_time_td"] = extract_timedelta(best_eff_df["elapsed_time"])

best_eff_df[["start_date_utc_dt", "start_date_local_dt", "local_timezone"]] = create_datetime_tz_cols(best_eff_df, "start_date", "start_date_local")
best_eff_df = pd.merge(best_eff_df.drop(columns="achievements"), explode_normalize_json(best_eff_df, 'achievements', 'id'), on='id', how='left')

In [37]:
best_eff_df_cols_clean = [
    'id',
    'name',
    'start_date_utc_dt',
    'start_date_local_dt',
    'local_timezone',
    'distance',
    'moving_time',
    'moving_time_td',
    'elapsed_time',
    'elapsed_time_td',
    'rank',
    'type',
    'activity_id'
]
best_eff_df = best_eff_df[best_eff_df_cols_clean]

In [ ]:
best_eff_df.head()

In [ ]:
best_eff_df_dtype_map = {
    "id": BigInteger,
    "name": String,
    "start_date_utc_dt": DateTime(timezone=False),
    "start_date_local_dt": DateTime(timezone=False),
    "local_timezone": String,
    "distance": Float,
    "moving_time": Integer,
    "moving_time_td": Interval,
    "elapsed_time": Integer,
    "elapsed_time_td": Interval,
    "rank": Integer,
    "type": String,
    "activity_id": BigInteger
}

with engine.begin() as conn:
    conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {TARGET_S_SCHEMA};"))

logging.warning(f"Whole table {TARGET_S_SCHEMA}.{BEST_EFFORTS_S_TABLE} will be overwritten.")

best_eff_df.to_sql(
    name=BEST_EFFORTS_S_TABLE,
    schema=TARGET_S_SCHEMA,
    con=engine,
    if_exists="replace",
    index=False,
    dtype=best_eff_df_dtype_map,
    method="multi",
    chunksize=1000
)

## Kudos Dataframe

In [40]:
kudos_df['full_name'] = kudos_df[['firstname', 'lastname']].astype('string').agg(' '.join, axis=1)
kudos_df_cols_clean = [
    'firstname',
    'lastname',
    'full_name',
    'activity_id'
]
kudos_df = kudos_df[kudos_df_cols_clean]
kudos_df = kudos_df.rename(columns={'firstname': 'first_name', 'lastname': 'last_name'})

In [ ]:
kudos_df.head()

In [ ]:
kudos_dtype_map = {
    "firs_tname": String,
    "last_name": String,
    "full_name": String,
    "activity_id": BigInteger
}

with engine.begin() as conn:
    conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {TARGET_S_SCHEMA};"))

logging.warning(f"Whole table {TARGET_S_SCHEMA}.{KUDOS_S_TABLE} will be overwritten.")

kudos_df.to_sql(
    name=KUDOS_S_TABLE,
    schema=TARGET_S_SCHEMA,
    con=engine,
    if_exists="replace",
    index=False,
    dtype=kudos_dtype_map,
    method="multi",
    chunksize=1000
)

## Activities Zones Dataframe

In [43]:
def zones_sort_add_names(bucket: list, zone_type: str) -> list:
  """
  Sorts a list of zone dicts by their 'min' boundary, assigns a 1-based
  'zone_number' to each entry, and sets a human-readable 'zone_name'
  based on the provided zone type.

  Behavior
  --------
  - The list is sorted ascending by the 'min' key.
  - Each zone dict is mutated in place by adding:
      * 'zone_number' (int): 1, 2, 3, ...
      * 'zone_name' (str): depends on `type` and the assigned number.
  - For type='heartrate':
      1→'Z1 - Recovery', 2→'Z2 - Endurance', 3→'Z3 - Tempo',
      4→'Z4 - Threshold', 5→'Z5 - Anaerobic'; other numbers → ''.
  - For type='pace':
      1→'Z1 - Recovery', 2→'Z2 - Endurance', 3→'Z3 - Tempo',
      4→'Z4 - Threshold', 5→'Z5 - VO2 Max', 6→'Z6 - Anaerobic';
      other numbers → ''.
  - Any other `type` produces an empty 'zone_name' for all entries.

  Parameters
  ----------
  bucket : list[dict]
      List of dictionaries representing zones. Each dict must contain
      a numeric 'min' key used for sorting.
      NOTE: The dictionaries are modified in place.
  type : str
      Zone classification to use for naming. Expected values:
      'heartrate' or 'pace'. Others fall back to empty names.

  Returns
  -------
  list[dict]
      A new list with the same dict objects, sorted by 'min' and
      with 'zone_number' and 'zone_name' added.

  Raises
  ------
  KeyError
      If any dict is missing the 'min' key.
  TypeError
      If 'min' values are not comparable (e.g., non-numeric types).
  """

  if zone_type not in ['heartrate', 'pace']:
        raise ValueError(f"Unsupported zone_type: {zone_type!r}")

  sorted_bucket = sorted(bucket, key=lambda d: d['min'])

  for i, zone in enumerate(sorted_bucket):
    zone['zone_number'] = i + 1

    if zone_type == 'heartrate':
      match zone['zone_number']:
        case 1:
          zone['zone_name'] = 'Z1 - Recovery'
        case 2:
          zone['zone_name'] = 'Z2 - Endurance'
        case 3:
          zone['zone_name'] = 'Z3 - Tempo'
        case 4:
          zone['zone_name'] = 'Z4 - Threshold'
        case 5:
          zone['zone_name'] = 'Z5 - Anaerobic'
        case _:
          zone['zone_name'] = ''
    elif zone_type == 'pace':
      match zone['zone_number']:
        case 1:
          zone['zone_name'] = 'Z1 - Recovery'
        case 2:
          zone['zone_name'] = 'Z2 - Endurance'
        case 3:
          zone['zone_name'] = 'Z3 - Tempo'
        case 4:
          zone['zone_name'] = 'Z4 - Threshold'
        case 5:
          zone['zone_name'] = 'Z5 - VO2 Max'
        case 6:
          zone['zone_name'] = 'Z6 - Anaerobic'
        case _:
          zone['zone_name'] = ''
    else:
      zone['zone_name'] = ''
  
  return sorted_bucket

In [ ]:
zones_df = activities_zones_df.copy()

In [45]:
zones_df = zones_df[zones_df['type'].isin(['heartrate', 'pace'])]

In [46]:
zones_df['distribution_buckets_clean'] = zones_df.apply(
  lambda row: zones_sort_add_names(row['distribution_buckets'], row['type']), axis=1)
zones_df = pd.merge(zones_df.drop(columns="distribution_buckets_clean"), explode_normalize_json(zones_df, 'distribution_buckets_clean', 'id'), on='id', how='left')
zones_df['id'] = zones_df[['activity_id', 'type', 'zone_number']].astype('string').agg('-'.join, axis=1)


In [ ]:
zones_df.head(15)

In [48]:
zones_df_cols_clean = [
    'id',
    'activity_id',
    'type',
    'zone_number',
    'zone_name',
    'time',
    'min',
    'max'
]
zones_df = zones_df[zones_df_cols_clean]

In [ ]:
zones_df.head()

In [ ]:
zones_dtype_map = {
    "id": String,
    "activity_id": BigInteger,
    "type": String,
    'zone_number': Integer,
    "zone_name": String,
    "time": Float
    
}

with engine.begin() as conn:
    conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {TARGET_S_SCHEMA};"))

logging.warning(f"Whole table {TARGET_S_SCHEMA}.{ZONES_S_TABLE} will be overwritten.")

zones_df.to_sql(
    name=ZONES_S_TABLE,
    schema=TARGET_S_SCHEMA,
    con=engine,
    if_exists="replace",
    index=False,
    dtype=zones_dtype_map,
    method="multi",
    chunksize=1000
)

## Relative Effort Dataframe

In [51]:
relative_effort_df = activities_zones_df.copy()

In [52]:
relative_effort_df = relative_effort_df[relative_effort_df['type'] == 'heartrate']
relative_effort_df = relative_effort_df[['activity_id', 'score']]
relative_effort_df = relative_effort_df.rename(columns={'score': 'relative_effort'})

In [ ]:
relative_effort_df.head()

In [ ]:
relative_effort_dtype_map = {
    "activity_id": BigInteger,
    "relative_effort": Float
}

with engine.begin() as conn:
    conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {TARGET_S_SCHEMA};"))

logging.warning(f"Whole table {TARGET_S_SCHEMA}.{RELATIVE_EFFORT_S_TABLE} will be overwritten.")

relative_effort_df.to_sql(
    name=RELATIVE_EFFORT_S_TABLE,
    schema=TARGET_S_SCHEMA,
    con=engine,
    if_exists="replace",
    index=False,
    dtype=relative_effort_dtype_map,
    method="multi",
    chunksize=1000
)

## Decoding coordinates with geopy

### Create list of unique locations

In [55]:
lat_lng_points_df = pd.concat([activities_df[['start_lat', 'start_lng']], segments_df[['start_lat', 'start_lng']]]).drop_duplicates()

In [56]:
geo = Nominatim(user_agent=USER_AGENT)
reverse = RateLimiter(geo.reverse, min_delay_seconds=1.1, max_retries=3, error_wait_seconds=5)

def cache_key(lat, lng, precision):
  """
  Create a stable cache key for a coordinate cell.

  Parameters
  ----------
  lat : float
      Latitude in decimal degrees.
  lng : float
      Longitude in decimal degrees.
  precision : int
      Number of decimal places used to format the coordinates.
      This effectively defines the grid cell size (e.g., 2 dp ≈ city level).

  Returns
  -------
  str
      Key formatted as ``"lat,lng"`` with fixed precision, e.g. ``"51.11,17.02"``.
      Using a string avoids floating-point representation issues and works as a JSON key.
  """

  return f"{lat:.{precision}f},{lng:.{precision}f}"

def pick_locality(addr):
  """
  Select a locality (city/town/village) from a Nominatim ``address`` mapping.

  The function returns the first non-empty value in the following order of preference:
  ``city`` → ``town`` → ``village`` → ``hamlet`` → ``municipality`` → ``locality`` → ``county`` (fallback).

  Parameters
  ----------
  addr : Mapping[str, str]
      The ``address`` object from a Nominatim response (``loc.raw['address']``).

  Returns
  -------
  str or None
      Locality name or ``None`` if none of the keys are available.

  Notes
  -----
  Including ``county`` as the last-resort fallback may return a county name
  in places where a true locality is missing in OSM data (useful in some countries),
  but it can be more general than a town/city.
  """
  
  return (addr.get("city") or addr.get("town") or addr.get("village") or addr.get("hamlet") or addr.get("municipality") or addr.get("locality") or addr.get("county"))

def pick_region(addr):
  """
  Select a region/state from a Nominatim ``address`` mapping.

  The function returns the first non-empty value in the following order of preference:
  ``state`` → ``region`` → ``state_district`` → ``province`` → ``county`` (fallback).

  Parameters
  ----------
  addr : Mapping[str, str]
      The ``address`` object from a Nominatim response (``loc.raw['address']``).

  Returns
  -------
  str or None
      Region/administrative area name or ``None`` if not present.

  Notes
  -----
  In Poland and many countries ``state`` corresponds to the top-level region
  (e.g., voivodeship/province). ``county`` is typically a lower level and is used
  here only as a fallback for countries where counties act as primary regions.
  """
  
  return (addr.get("state") or addr.get("region") or addr.get("state_district") or addr.get("province") or addr.get("county"))

def address_fields(reverse_fn, lat, lng):
  """
  Perform reverse geocoding and extract minimal address fields.

  Parameters
  ----------
  reverse_fn : Callable
      A function compatible with ``geopy.Nominatim.reverse`` (optionally wrapped
      with ``RateLimiter``) that accepts ``(lat, lng)`` and returns a Location-like
      object with a ``.raw`` dict payload.
  lat : float
      Latitude in decimal degrees.
  lng : float
      Longitude in decimal degrees.

  Returns
  -------
  dict
      Dictionary with three keys:
      - ``locality`` : str or None — city/town/village (best available),
      - ``region``   : str or None — region/state/province,
      - ``country``  : str or None — country name.

  Notes
  -----
  The function calls ``reverse_fn`` with ``language="en"`` and ``addressdetails=True``.
  Change the language parameter if localized names are desired.
  """

  loc = reverse_fn((lat, lng), language="en", addressdetails=True)
  if not loc:
      return {"locality": None, "region": None, "country": None}
  address = (loc.raw or {}).get("address", {})
  return {
      "locality": pick_locality(address),
      "region": pick_region(address),
      "country": address.get("country")
  }

def decode_coordinates(coordinates_df: pd.DataFrame, lat_col: str, lng_col: str, cache_path: str, precision: int, geo_reverse_fn) -> pd.DataFrame:
  """
  Append ``locality``, ``region``, and ``country`` to a DataFrame using reverse geocoding with caching.

  For each unique coordinate pair (rounded to ``precision`` decimal places), the function performs
  a reverse geocode via ``geo_reverse_fn`` and stores results in a JSON cache. Subsequent runs read
  from the cache to minimize API calls.

  Parameters
  ----------
  coordinates_df : pd.DataFrame
      Input DataFrame containing coordinate columns.
  lat_col : str
      Name of the latitude column (e.g., ``"start_lat"``).
  lng_col : str
      Name of the longitude column (e.g., ``"start_lng"``).
  cache_path : str
      File path to the JSON cache. The file will be created/updated as needed.
  precision : int
      Number of decimal places for rounding coordinates and building the cache key
      (e.g., 2 ≈ city-level granularity).
  geo_reverse_fn : Callable
      Reverse geocoding function (typically a ``RateLimiter(Nominatim.reverse, ...)``).

  Returns
  -------
  pd.DataFrame
      A copy of the input DataFrame with three additional columns:
      ``locality``, ``region``, and ``country``.

  Raises
  ------
  KeyError
      If the required ``lat_col`` or ``lng_col`` is missing from ``coordinates_df``.
  json.JSONDecodeError
      If the cache file exists but contains invalid JSON.
  Exception
      Any exception propagated from the reverse geocoding function or file I/O.

  Notes
  -----
  - The function writes the cache to ``cache_path`` whenever a new key is added.
  - A small sleep (``time.sleep(0.5)``) is used per new lookup; adjust to comply with
    your provider's rate limits (public Nominatim typically requires ≤1 request/sec).
  - Change the language in ``address_fields`` if you need localized names.
  """

  temp_df = coordinates_df.copy()

  if lat_col not in temp_df.columns or lng_col not in temp_df.columns:
    raise KeyError(f"Missing required columns: {lat_col}, {lng_col}")
  
  temp_df['lat_round'] = temp_df[lat_col].round(precision)
  temp_df['lng_round'] = temp_df[lng_col].round(precision)
  temp_df = temp_df.dropna(subset=[lat_col, lng_col])
  lat_lng_df = temp_df[['lat_round', 'lng_round']].drop_duplicates().reset_index(drop=True)

  cache = {}
  if os.path.exists(cache_path):
    with open(cache_path, "r", encoding="utf-8") as f:
      cache = json.load(f)

  records = []
  for i, row in tqdm(lat_lng_df.iterrows(), total=lat_lng_df.shape[0]):

    key = cache_key(row['lat_round'], row['lng_round'], precision)

    if key in cache:
      resp = cache[key]

    else:
      resp = address_fields(geo_reverse_fn, row['lat_round'], row['lng_round'])
      cache[key] = resp
      time.sleep(0.5)

      with open(cache_path, "w", encoding="utf-8") as f:
        json.dump(cache, f, ensure_ascii=False, indent=2)

    records.append({"lat_round": row['lat_round'], "lng_round": row['lng_round'], **resp})

  loc_df = pd.DataFrame(records)
  
  result = temp_df.merge(loc_df, on=["lat_round","lng_round"], how="left")
  result = result.drop(columns=["lat_round","lng_round"])

  return result

In [ ]:
lat_lng_points_df = decode_coordinates(lat_lng_points_df, 'start_lat', 'start_lng', CACHE_PATH, PRECISION, reverse)

In [ ]:
lat_lng_points_df.head()

In [59]:
locations_df = lat_lng_points_df[['locality', 'region', 'country']].copy()
locations_df = locations_df.drop_duplicates()
locations_df = locations_df.sort_values(by=['country', 'region', 'locality']).reset_index(drop=True)
locations_df['location_id'] = 1000 + np.arange(len(locations_df))
locations_df = locations_df[['location_id', 'country', 'region', 'locality']]

In [ ]:
locations_df.head()

In [61]:
lat_lng_points_df = pd.merge(lat_lng_points_df, locations_df, how='left', on=['country', 'region', 'locality'])

In [ ]:
lat_lng_points_df.head()

In [63]:
activities_df = pd.merge(activities_df, lat_lng_points_df[['start_lat', 'start_lng', 'location_id']], how='left', on=['start_lat', 'start_lng'])

In [ ]:
activities_df.head()

In [ ]:
activities_df_dtype_map = {
    "id": BigInteger,
    "name": String,
    "start_date_utc_dt": DateTime(timezone=False),
    "start_date_local_dt": DateTime(timezone=False),
    "local_timezone":  String,
    "distance": Float,
    "moving_time": Integer,
    "moving_time_td": Interval,
    "elapsed_time": Integer,
    "elapsed_time_td": Interval,
    "total_elevation_gain": Float,
    "elev_low": Float,
    "elev_high": Float,
    "type": String,
    "sport_type": String,
    "workout_type": Integer,
    "achievement_count": Integer,
    "kudos_count": Integer,
    "comment_count": Integer,
    "athlete_count": Integer,
    "photo_count": Integer,
    "trainer": Boolean,
    "commute": Boolean,
    "manual": Boolean,
    "visibility": String,
    "average_speed": Float,
    "avg_pace_str": String,
    "avg_pace_float": Float,
    "max_speed": Float,
    "max_pace_str": String,
    "max_pace_float": Float,
    "average_cadence": Float,
    "average_watts": Float,
    "max_watts": Float,
    "weighted_average_watts": Float,
    "has_heartrate": Boolean,
    "average_heartrate": Float,
    "max_heartrate": Float,
    "pr_count": Integer,
    "total_photo_count": Integer,
    "suffer_score": Float,
    "description": Text,
    "calories": Float,
    "device_name": String,
    'start_lat' : Float,
    'start_lng' : Float,
    "map_id": String,
    "gear_id": String,
    "location_id": Integer
}

with engine.begin() as conn:
    conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {TARGET_S_SCHEMA};"))

logging.warning(f"Whole table {TARGET_S_SCHEMA}.{ACTIVITIES_S_TABLE} will be overwritten.")

activities_df.to_sql(
    name=ACTIVITIES_S_TABLE,
    schema=TARGET_S_SCHEMA,
    con=engine,
    if_exists="replace",
    index=False,
    dtype=activities_df_dtype_map,
    method="multi",
    chunksize=1000
)

In [ ]:
activities_df.dtypes

In [67]:
segments_df = pd.merge(segments_df, lat_lng_points_df[['start_lat', 'start_lng',  'location_id']], how='left', on=['start_lat', 'start_lng'])

In [ ]:
segments_df.head()

In [ ]:
segments_df_dtype_map = {
    "id": BigInteger,
    "name": String,
    "activity_type": String,
    "distance": Float,
    "average_grade": Float,
    "maximum_grade": Float,
    "elevation_high": Float,
    "elevation_low": Float,
    "elevation_profile": Float,
    "elevation_profiles": Float,
    "climb_category": Float,
    "private": Boolean,
    "hazardous": Boolean,
    "starred": Boolean,
    "start_lat": Float,
    "start_lng": Float,
    "end_lat": Float,
    "end_lng": Float,
    "location_id": Integer
}

with engine.begin() as conn:
    conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {TARGET_S_SCHEMA};"))

logging.warning(f"Whole table {TARGET_S_SCHEMA}.{SEGMENTS_S_TABLE} will be overwritten.")

segments_df.to_sql(
    name=SEGMENTS_S_TABLE,
    schema=TARGET_S_SCHEMA,
    con=engine,
    if_exists="replace",
    index=False,
    dtype=segments_df_dtype_map,
    method="multi",
    chunksize=1000
)

In [70]:
locations_df = locations_df.rename(columns={'location_id': 'id'})

In [ ]:
locations_df.head()

In [ ]:
locations_df_dtype_map = {
    "id": Integer,
    "locality": String,
    "region	": String,
    "country": String
}

with engine.begin() as conn:
    conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {TARGET_S_SCHEMA};"))

logging.warning(f"Whole table {TARGET_S_SCHEMA}.{LOCATIONS_S_TABLE} will be overwritten.")

locations_df.to_sql(
    name=LOCATIONS_S_TABLE,
    schema=TARGET_S_SCHEMA,
    con=engine,
    if_exists="replace",
    index=False,
    dtype=locations_df_dtype_map,
    method="multi",
    chunksize=1000
)

### Primary keys definition

In [73]:
keys_instructions= [
    f"""CREATE SCHEMA IF NOT EXISTS {TARGET_S_SCHEMA};""",
    # ********** PRIMARY KEYS **********
    # --- activities ---
    # PK
    f"""
    DO $$
    BEGIN
      IF to_regclass('{TARGET_S_SCHEMA}.{ACTIVITIES_S_TABLE}') IS NOT NULL
         AND NOT EXISTS (
           SELECT 1 FROM pg_constraint
           WHERE conrelid = to_regclass('{TARGET_S_SCHEMA}.{ACTIVITIES_S_TABLE}')
             AND contype = 'p'
         )
      THEN
        ALTER TABLE {TARGET_S_SCHEMA}.{ACTIVITIES_S_TABLE}
          ADD CONSTRAINT {ACTIVITIES_S_TABLE}_pkey PRIMARY KEY (id);
      END IF;
    END $$;
    """,
    # --- gear ---
    # PK
    f"""
    DO $$
    BEGIN
      IF to_regclass('{TARGET_S_SCHEMA}.{GEAR_S_TABLE}') IS NOT NULL
         AND NOT EXISTS (
           SELECT 1 FROM pg_constraint
           WHERE conrelid = to_regclass('{TARGET_S_SCHEMA}.{GEAR_S_TABLE}')
             AND contype = 'p'
         )
      THEN
        ALTER TABLE {TARGET_S_SCHEMA}.{GEAR_S_TABLE}
          ADD CONSTRAINT {GEAR_S_TABLE}_pkey PRIMARY KEY (id);
      END IF;
    END $$;
    """,
    # --- segments ---
    # PK
    f"""
    DO $$
    BEGIN
      IF to_regclass('{TARGET_S_SCHEMA}.{SEGMENTS_S_TABLE}') IS NOT NULL
         AND NOT EXISTS (
           SELECT 1 FROM pg_constraint
           WHERE conrelid = to_regclass('{TARGET_S_SCHEMA}.{SEGMENTS_S_TABLE}')
             AND contype = 'p'
         )
      THEN
        ALTER TABLE {TARGET_S_SCHEMA}.{SEGMENTS_S_TABLE}
          ADD CONSTRAINT {SEGMENTS_S_TABLE}_pkey PRIMARY KEY (id);
      END IF;
    END $$;
    """,
    # --- laps ---
    # PK
    f"""
    DO $$
    BEGIN
      IF to_regclass('{TARGET_S_SCHEMA}.{LAPS_S_TABLE}') IS NOT NULL
         AND NOT EXISTS (
           SELECT 1 FROM pg_constraint
           WHERE conrelid = to_regclass('{TARGET_S_SCHEMA}.{LAPS_S_TABLE}')
             AND contype = 'p'
         )
      THEN
        ALTER TABLE {TARGET_S_SCHEMA}.{LAPS_S_TABLE}
          ADD CONSTRAINT {LAPS_S_TABLE}_pkey PRIMARY KEY (id);
      END IF;
    END $$;
    """,
    # --- best efforts ---
    # PK
    f"""
    DO $$
    BEGIN
      IF to_regclass('{TARGET_S_SCHEMA}.{BEST_EFFORTS_S_TABLE}') IS NOT NULL
         AND NOT EXISTS (
           SELECT 1 FROM pg_constraint
           WHERE conrelid = to_regclass('{TARGET_S_SCHEMA}.{BEST_EFFORTS_S_TABLE}')
             AND contype = 'p'
         )
      THEN
        ALTER TABLE {TARGET_S_SCHEMA}.{BEST_EFFORTS_S_TABLE}
          ADD CONSTRAINT {BEST_EFFORTS_S_TABLE}_pkey PRIMARY KEY (id);
      END IF;
    END $$;
    """,
    # --- locations ---
    # PK
    f"""
    DO $$
    BEGIN
      IF to_regclass('{TARGET_S_SCHEMA}.{LOCATIONS_S_TABLE}') IS NOT NULL
         AND NOT EXISTS (
           SELECT 1 FROM pg_constraint
           WHERE conrelid = to_regclass('{TARGET_S_SCHEMA}.{LOCATIONS_S_TABLE}')
             AND contype = 'p'
         )
      THEN
        ALTER TABLE {TARGET_S_SCHEMA}.{LOCATIONS_S_TABLE}
          ADD CONSTRAINT {LOCATIONS_S_TABLE}_pkey PRIMARY KEY (id);
      END IF;
    END $$;
    """,
    # --- zones ---
    # PK
    f"""
    DO $$
    BEGIN
      IF to_regclass('{TARGET_S_SCHEMA}.{ZONES_S_TABLE}') IS NOT NULL
         AND NOT EXISTS (
           SELECT 1 FROM pg_constraint
           WHERE conrelid = to_regclass('{TARGET_S_SCHEMA}.{ZONES_S_TABLE}')
             AND contype = 'p'
         )
      THEN
        ALTER TABLE {TARGET_S_SCHEMA}.{ZONES_S_TABLE}
          ADD CONSTRAINT {ZONES_S_TABLE}_pkey PRIMARY KEY (id);
      END IF;
    END $$;
    """,
    # --- relative_effort ---
    # PK
    f"""
    DO $$
    BEGIN
      IF to_regclass('{TARGET_S_SCHEMA}.{RELATIVE_EFFORT_S_TABLE}') IS NOT NULL
         AND NOT EXISTS (
           SELECT 1 FROM pg_constraint
           WHERE conrelid = to_regclass('{TARGET_S_SCHEMA}.{RELATIVE_EFFORT_S_TABLE}')
             AND contype = 'p'
         )
      THEN
        ALTER TABLE {TARGET_S_SCHEMA}.{RELATIVE_EFFORT_S_TABLE}
          ADD CONSTRAINT {RELATIVE_EFFORT_S_TABLE}_pkey PRIMARY KEY (activity_id);
      END IF;
    END $$;
    """
]

### Create constrains

In [74]:
with engine.begin() as conn:
    for sql in keys_instructions:
        conn.execute(text(sql))